In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pylab
from scipy import stats
import glob
import os
import shutil

In [2]:
#Function to identify the subject index of a given subject name
def FindSubIdx(name,all_names):
    for el,nn in enumerate(all_names):
        if(nn==name):
            return el

# Load Human FFF data

In [3]:
##### Load prepared Excel data file with scores
GameNb=15
SubjNb = 41
names=[j for j in range(SubjNb)]
conds=[('','') for j in range(SubjNb)]
avgs=np.zeros(SubjNb)
SeqFile = '../Inputs/Humans_Part_I/Fast/Summary_Scores_FFF.txt'
f_toOpen = open(SeqFile,"r") 
lines = [line.rstrip().split("\t") for line in f_toOpen]
cats = lines[0]
print(cats)
print(lines[1])
idx=0
for ii in np.arange(1,len(lines)):
    cur_line=lines[ii]
    names[idx] = cur_line[0]
    conds[idx] = (cur_line[1],cur_line[2])
    idx=idx+1
print(names)
print(conds)

['Participant', 'condition1', 'condition2', 'Game 1', 'Game 2', 'Game 3', 'Game 4', 'Game 5', 'Game 6', 'Game 7', 'Game 8', 'Game 9', 'Game 10', 'Game 11', 'Game 12', 'Game 13', 'Game 14', 'Game 15']
['A11LNK1U3DT08V', 'fast', 'fast', '526', '1626', '1604', '1640', '1408', '1640', '1840', '1844', '1656', '1638', '1644', '1948', '1860', '1956', '1734']
['A11LNK1U3DT08V', 'A13B8594JIUFDF', 'A13G6IRFQBEE8K', 'A19X8IA9EKC3XH', 'A1DHJE69R494AH', 'A1FG2G0J8M9Q0N', 'A1U6L1L2CUQNU5', 'A25W2106F90G7L', 'A2DMUU9GE8QA5Q', 'A2FM6APZP3FN22', 'A2PW5825FIRONK', 'A376AZWIBALRC', 'A38V6MB0UIXY0N', 'A3D1QQ8BEO75C6', 'A3H2KP0VBENV3N', 'A3LNWWZ7HGAGGM', 'A3Q4Y5ONYPSQGF', 'A7OZPNXIVO1FX', 'AJZYNIL3ITOK0', 'AUPXYEKITX8UT', 'AY4PZPXHSRCP5', 'A1EP3XESXAG0EP', 'A2RY4MK1FONYH1', 'A2W8TJXW68442D', 'AAE1COF4E8740', 'AY5OS6NZWVX22', 'A122LRCSBAD6DC', 'A1RH69F7I73LRW', 'A1RV2LERVS0A4H', 'A2BFFKEYMCBFWW', 'A3LA0JBSIW4MUI', 'A13YOOKZFZJJ10', 'A1SISJL5ST2PWH', 'A1U2A42K8FV3RA', 'A3IGG3RX4HXUV9', 'AFM65NU0UXIGP', 'AQXR

In [4]:
GameNb=15
MeasureN = 10
AgentN = 41
cond_names = {('fast', 'fast'):'FFF'}
cond_idx = {'FFF':0}
list_speeds = ['FFF']

def getList(dict):
    return dict.keys()

In [5]:
#Deal with fast speed first
measures = {'Score':0,'Entropy':1,'Amplitude':2,'Periodicity':3,'Deflations':4,'Misses':5,'Resets':6,'Right_Amplitude':7,'LogCV':8}
Map_File = {"AllDeflations.txt":'Deflations',"AllMisses.txt":'Misses',
                "AllResets.txt":'Resets',"Amplitude_Transfer_Humans_SORTED.txt":'Amplitude',
                "Entropy_Transfer_Humans_SORTED.txt":'Entropy',"logCVIFI_Transfer_Humans.txt":'LogCV',
                "Periodicity_Transfer_Humans_SORTED.txt":'Periodicity',"RightAmp_Transfer_Humans_SORTED.txt":'Right_Amplitude',
                "RightPer_Transfer_Humans_SORTED.txt":'Right-Periodicity',
                "Summary_Scores_FFF.txt":'Score'}
List_Files = list(getList(Map_File))
print(List_Files)
pb_idx = [3,4,5,6,7,8] #problem indices

['AllDeflations.txt', 'AllMisses.txt', 'AllResets.txt', 'Amplitude_Transfer_Humans_SORTED.txt', 'Entropy_Transfer_Humans_SORTED.txt', 'logCVIFI_Transfer_Humans.txt', 'Periodicity_Transfer_Humans_SORTED.txt', 'RightAmp_Transfer_Humans_SORTED.txt', 'RightPer_Transfer_Humans_SORTED.txt', 'Summary_Scores_FFF.txt']


In [6]:
import os 
#current dir
cwd = os.getcwd()
print(cwd)

#Open a new file and write to file every single line (not last line if empty): make sure to use a different name
cur_path = cwd
HumanName = 'Human_FFF'
print(HumanName)
new_file = "./R_Data_"+HumanName+".txt"
print(new_file)
cfile = open(new_file,"w+")
ToWrite = "Type\tSpeed\tAgent\tsubject\tGameNb\tDeflations\tMisses\tResets\tShotReg\tEntropy\tLogCV\tShotPer\tRightReg\tRightPer\tScore\n"
cfile.write(ToWrite)

C:\Users\pierregianferrara\Documents\Academic\CMU_PhD\Research\Papers\Paper2_2022\KiltHub_v2\Code\Analyses_Part_I
Human_FFF
./R_Data_Human_FFF.txt


111

In [7]:
#loop across speeds here
#print(ModelName)
AllData = np.zeros((MeasureN,AgentN,GameNb))
names = []

for cIDX,curFile in enumerate(List_Files):
    measure_File= '../Inputs/Humans_Part_I/Fast/'+curFile
    print(measure_File)
    thisfile = open(measure_File,"r")
    AllLines = [line.rstrip().split("\t") for line in thisfile]
    this_data = np.array([[float(ee) for ee in rr] for rr in [el[-15:] for el in AllLines[1:]]]) #read lines in file
    if(cIDX==0):
        names = [el[0] for el in AllLines[1:]]
    thisfile.close()
    this_Measure = Map_File[curFile]
    cur_speed = 'FFF'
    
    #Get all means and standard deviations and write to file
    if(not(cIDX in pb_idx)):
        #add to data
        for nn in range(AgentN):
            line_data = this_data[nn]
            c_name = AllLines[1:][nn][0] #get this specific name
            nameIDX = FindSubIdx(c_name,names) #find corresponding index
            for gg in range(GameNb):
                AllData[cIDX,nameIDX,gg] = line_data[gg]
    else:
        if(cIDX==4):
            this_data[this_data>4.70] = np.nan
            #add to data
            for nn in range(AgentN):
                line_data = this_data[nn]
                c_name = AllLines[1:][nn][0] #get this specific name
                nameIDX = FindSubIdx(c_name,names) #find corresponding index
                for gg in range(GameNb):
                    AllData[cIDX,nameIDX,gg] = line_data[gg]
        else:
            this_data[this_data==0.0] = np.nan
            #add to data
            for nn in range(AgentN):
                line_data = this_data[nn]
                c_name = AllLines[1:][nn][0] #get this specific name
                nameIDX = FindSubIdx(c_name,names) #find corresponding index
                for gg in range(GameNb):
                    AllData[cIDX,nameIDX,gg] = line_data[gg]

for nn in range(AgentN):
    for gg in range(GameNb):
        #Write to File
        ToWrite = "Human\tFFF\t"+HumanName+'\t'+names[nn]+'\t'+str(gg+1)
        for iii in range(MeasureN):
            ToWrite = ToWrite + '\t' + str(AllData[iii,nn,gg])
        ToWrite = ToWrite + '\n'
        cfile.write(ToWrite)
        
#Finally, close file when done
cfile.close()

../Inputs/Humans_Part_I/Fast/AllDeflations.txt
../Inputs/Humans_Part_I/Fast/AllMisses.txt
../Inputs/Humans_Part_I/Fast/AllResets.txt
../Inputs/Humans_Part_I/Fast/Amplitude_Transfer_Humans_SORTED.txt
../Inputs/Humans_Part_I/Fast/Entropy_Transfer_Humans_SORTED.txt
../Inputs/Humans_Part_I/Fast/logCVIFI_Transfer_Humans.txt
../Inputs/Humans_Part_I/Fast/Periodicity_Transfer_Humans_SORTED.txt
../Inputs/Humans_Part_I/Fast/RightAmp_Transfer_Humans_SORTED.txt
../Inputs/Humans_Part_I/Fast/RightPer_Transfer_Humans_SORTED.txt
../Inputs/Humans_Part_I/Fast/Summary_Scores_FFF.txt
